In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../..'))


# from scan_engine.status_fields_config_24summ import STATUS_FIELDS_CONFIG
from status_fields_config_oct3v1_brws_250522 import STATUS_FIELDS_CONFIG

from utlis.scan_engine_utlis.scan_eng_big_utlis import log_folder_to_parquet_sep

if __name__ == "__main__":
    base_folder = "/data/big_rim/rsync_dcc_sum/Oct3V1" #"/data/big_rim/rsync_dcc_sum/25Apri_social"
    # "/data/big_rim/rsync_dcc_sum/24summ" #"/data/big_rim/rsync_dcc_sum/25Apri_social" #"/data/big_rim/rsync_dcc_sum/Oct3V1" #"/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"  # Replace with your base folder
    # save_path = os.path.join(base_folder, 'paret')
    failed_paths_file = "/data/big_rim/rsync_dcc_sum/Oct3V1/sync_failed_brws.txt"  # File containing failed paths


    force_rescan_rec_files = [
        # ('2023-10-01', '001'),
        # ('2023-10-02', '002'),
        # Add more as needed
    ]
    rescan_threshold_days = 0.001 # 7 days, but guess if i mess up i can just change it to automatically rescan all, smile... #0.1

    log_folder_to_parquet_sep(base_folder, failed_paths_file, STATUS_FIELDS_CONFIG,
                              force_rescan_rec_files=force_rescan_rec_files,
                              rescan_threshold_days=rescan_threshold_days)


Log for 20241217v1l23re1_p20241217v1l23BE0 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_21/20241217v1l23re1_p20241217v1l23BE0/folder_log.parquet
Log for 20241217v1l23re1 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_21/20241217v1l23re1/folder_log.parquet
Log for 20241212V1RE1L23F saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_04_10/20241212V1RE1L23F/folder_log.parquet
Log for 20241212V1RE1L23Fmini_p20241212V1BE1L23F saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_04_10/20241212V1RE1L23Fmini_p20241212V1BE1L23F/folder_log.parquet
Log for 20241212V1RE1L23Fmini_p20241212V1LE1L23F saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_04_10/20241212V1RE1L23Fmini_p20241212V1LE1L23F/folder_log.parquet
Log for 20241224PMCLE1_14_12 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_03_11/20241224PMCLE1_14_12/folder_log.parquet
Log for 20241224PMCLE1mini_p20240303PMC1_14_21 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_03_11/20241224PMCLE1mini_p20240303PMC1_14_21/folder_log.parquet
Lo

In [5]:
# sys.path.append(os.path.abspath('../..'))
from utlis.scan_engine_utlis.scan_engine_utlis import read_all_parquet_files
# base_folder = "/data/big_rim/rsync_dcc_sum/Oct3V1" 
all_df = read_all_parquet_files(base_folder)

In [9]:
import os
import glob

def validate_and_map_calibs(rec_paths):
    """
    For each rec_path, find the first *label3d_dannce.mat file,
    strip off any duplicated prefixes, and flag only those files
    whose names contain 'df_synced_' at least twice.

    Returns:
      mapping: dict where mapping[rec_path] = clean_filename or None
      wrong:   list of (rec_path, full_path) tuples for any files
               with two or more 'df_synced_' occurrences
    """
    mapping = {}
    wrong = []

    for entry in rec_paths:
        dir_path = entry[0] if isinstance(entry, (list, tuple)) else entry

        # locate the label3d_dannce file
        pattern = os.path.join(dir_path, "*label3d_dannce.mat")
        matches = glob.glob(pattern)

        if not matches:
            mapping[dir_path] = None
            continue

        full_path = matches[0]
        fname = os.path.basename(full_path)
        rec_folder = os.path.basename(dir_path)

        # only flag as wrong if 'df_synced_' appears two or more times
        if fname.count('df_synced_') > 1:
            wrong.append((dir_path, full_path))

        # produce a clean name by stripping everything before
        # the last occurrence of the folder name
        if rec_folder in fname:
            last_idx = fname.rfind(rec_folder)
            clean = fname[last_idx:]
        else:
            clean = fname

        mapping[dir_path] = clean

    return mapping, wrong

# --- usage example ---
rec_paths = all_df.column("rec_path").to_pylist()
rec_paths = [p for p in rec_paths if p is not None]
mapping, wrong_entries = validate_and_map_calibs(rec_paths)


In [ ]:
#below thing will jsut remove everything with sort of bad naming lol rip

# import os
# import glob

# def validate_and_map_calibs(rec_paths):
#     """
#     For each rec_path, find the first *label3d_dannce.mat file,
#     strip off any duplicated prefixes, and detect buggy repeats.

#     Returns:
#       mapping: dict where mapping[rec_path] = clean_filename or None
#       wrong:   list of (rec_path, full_path) tuples for any repeats found
#     """
#     mapping = {}
#     wrong = []

#     for entry in rec_paths:
#         # support either [path] or plain string
#         dir_path = entry[0] if isinstance(entry, (list, tuple)) else entry

#         # find any label3d_dannce.mat under that folder
#         pattern = os.path.join(dir_path, "*label3d_dannce.mat")
#         matches = glob.glob(pattern)

#         if not matches:
#             mapping[dir_path] = None
#             continue

#         full_path = matches[0]
#         fname     = os.path.basename(full_path)
#         rec_folder= os.path.basename(dir_path)

#         # count how many times the folder name appears
#         occ = fname.count(rec_folder)
#         if occ > 1:
#             # buggy repetition detected
#             wrong.append((dir_path, full_path))

#         # strip everything before the **last** occurrence of rec_folder
#         if rec_folder in fname:
#             last_idx = fname.rfind(rec_folder)
#             clean = fname[last_idx:]
#         else:
#             clean = fname

#         mapping[dir_path] = clean

#     return mapping, wrong

# # --- usage example ---
# # rec_paths = all_df.column("rec_path").to_pylist()
# rec_paths = [path for path in all_df.column("rec_path").to_pylist() if path is not None]

# mapping, wrong_entries = validate_and_map_calibs(rec_paths)


In [ ]:
import shutil

# # 1) see your full rec_path → clean filename
# for path, clean_name in mapping.items():
#     print(f"{path}  →  {clean_name}")

# 2) list out only the ones with repeated folder names
if wrong_entries:
    print("\nFound buggy repeats in these folders:")
    for path, full_path in wrong_entries:
        print(f" • {path}  →  {full_path}")

    # 3) if you’re ready to delete those bad files, uncomment below:
    for _, bad_file in wrong_entries:
        # os.remove(bad_file)
        # print("Deleted", bad_file)
        # shutil.rmtree(os.path.join(os.path.dirname(bad_file), 'prev_calib'), ignore_errors=True)
        # print("Deleted", os.path.join(os.path.dirname(bad_file), 'prev_calib'))
    #     shutil.rmtree(os.path.join(os.path.dirname(bad_file), 'COM'), ignore_errors=True)
    #     print("Deleted", os.path.join(os.path.dirname(bad_file), 'COM'))

    #     shutil.rmtree(os.path.join(os.path.dirname(bad_file), 'DANNCE'), ignore_errors=True)
    #     print("Deleted", os.path.join(os.path.dirname(bad_file), 'DANNCE'))
        
    #     shutil.rmtree(os.path.join(os.path.dirname(bad_file), 'MIR_Aligned'), ignore_errors=True)
    #     print("Deleted", os.path.join(os.path.dirname(bad_file), 'MIR_Aligned'))


Found buggy repeats in these folders:
 • /data/big_rim/rsync_dcc_sum/Oct3V1/2024_09_18/20240819_V1_r1  →  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_09_18/20240819_V1_r1/df_synced_20240819_V1_r1_2024_09_18_20240819_V1_r1_calib_before_label3d_dannce.mat
 • /data/big_rim/rsync_dcc_sum/Oct3V1/2024_09_18/20240819_V1_r1_2  →  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_09_18/20240819_V1_r1_2/df_synced_20240819_V1_r1_2_2024_09_18_20240819_V1_r1_2_calib_before_label3d_dannce.mat
 • /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03/20240819V1r2  →  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03/20240819V1r2/df_synced_20240819V1r2_2024_10_03_20240819V1r2_calib_before_label3d_dannce.mat
 • /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_bleach_2mice  →  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_bleach_2mice/df_synced_24Anshu_f_bleach_2mice_2024_10_03_micecolor_test_24Anshu_f_bleach_2mice_calib_before_label3d_dannce.mat
 • /data/big_rim/rsync_dcc_sum/Oct3V

In [7]:
import pyarrow.compute as pc
from functools import reduce

table = all_df  # combined_df
conditions = [
   pc.equal(table['dannce'], '1'),
   pc.equal(table['social'], '0'),
#    pc.equal(table['quality'], 'good_vein')
# quality_mask
]

filter_mask = reduce(pc.and_, conditions)
# Apply the filter and print the results
# for_dappy = table.filter(filter_mask) #filter_mask
for_social = table.filter(pc.equal(table['social'], '1'))
# Print each row of the filtered table
print(for_social.to_pandas())  # Display the filtered data in a familiar pandas-like format


   mir_generate_param sync mini_6cam_map dropf_handle com social miniscope  \
0                   1    1             0            0   0      1         1   
1                   1    1             0            0   0      1         1   
2                   1    1             0            0   0      1         1   
3                   1    1             0            0   0      1         1   
4                   1    1             0            0   0      1         1   
5                   1    1             0            0   0      1         1   
6                   1    1             0            0   0      1         1   
7                   1    1             0            0   0      1         1   
8                   1    1             0            0   0      1         1   
9                   1    1             0            0   0      1         1   
10                  1    1             1            0   0      1         1   
11                  1    1             1            0   0      1

In [ ]:
# import shutil

# chunked = for_social['rec_path']  

# # flatten out the list-of-lists into a simple list of strings
# # each element of chunked.to_pylist() is a list like ["/path/to/rec_folder"]
# path_lists = chunked.to_pylist()
# rec_paths = [lst[0] if isinstance(lst, list) else lst for lst in path_lists]

# # target base directory
# target_base = '/data/big_rim/rsync_dcc_sum/25socialmini'

# for orig in rec_paths:
#     # extract date folder and rec_folder names
#     date_folder = os.path.basename(os.path.dirname(orig))
#     rec_folder  = os.path.basename(orig)

#     # build destination path
#     dest = os.path.join(target_base, date_folder, rec_folder)

#     # ensure parent directory exists
#     os.makedirs(os.path.dirname(dest), exist_ok=True)

#     # copy only if not already present
#     if not os.path.exists(dest):
#         print(f'Copying:\n  {orig}\n→ {dest}\n')
#         shutil.copytree(orig, dest)
#     else:
#         print(f'Skipped (already exists): {dest}')

Copying:
  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_bleach_2mice
→ /data/big_rim/rsync_dcc_sum/25socialmini/2024_10_03_micecolor_test/24Anshu_f_bleach_2mice

Copying:
  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_paint_2mice
→ /data/big_rim/rsync_dcc_sum/25socialmini/2024_10_03_micecolor_test/24Anshu_f_paint_2mice

Copying:
  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_paint_2mice_2
→ /data/big_rim/rsync_dcc_sum/25socialmini/2024_10_03_micecolor_test/24Anshu_f_paint_2mice_2

Copying:
  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_29/2social_mini_0605pmc_single_18_00
→ /data/big_rim/rsync_dcc_sum/25socialmini/2024_10_29/2social_mini_0605pmc_single_18_00

Copying:
  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_29/2social_mini_0605pmc_single_18_25
→ /data/big_rim/rsync_dcc_sum/25socialmini/2024_10_29/2social_mini_0605pmc_single_18_25

Copying:
  /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_29/2social_mini_0605pm

In [9]:
import pyarrow.compute as pc
from functools import reduce

table = all_df  # combined_df
conditions = [
   pc.equal(table['dannce'], '1'),
   pc.equal(table['social'], '0'),
#    pc.equal(table['quality'], 'good_vein')
# quality_mask
]

filter_mask = reduce(pc.and_, conditions)
# Apply the filter and print the results
# for_dappy = table.filter(filter_mask) #filter_mask
for_dappy = table.filter(pc.equal(table['dannce'], '1'))
# Print each row of the filtered table
print(for_dappy.to_pandas())  # Display the filtered data in a familiar pandas-like format


   mir_generate_param sync dropf_handle com social miniscope test  \
0                   1    1            0   1      0         1    0   
1                   1    1            0   1      0         1    0   
2                   1    1            0   1      0         1    0   
3                   1    1            0   1      0         1    0   
4                   1    1            1   1      0         1    0   
..                ...  ...          ...  ..    ...       ...  ...   
64                  1    1            0   1      0         1    0   
65                  1    1            0   1      0         1    0   
66                  1    1            0   1      0         1    0   
67                  1    1            0   1      0         1    0   
68                  1    1            0   1      0         1    0   

   after_oxytocin before_oxytocin dannce mini_rec_sync  \
0               0               0      1             0   
1               0               0      1             0 

In [5]:
import pyarrow.compute as pc
table = all_df  # combined_df
filter_mask = pc.and_(
    # pc.equal(table['mir_generate_param'], '1'),
    pc.equal(table['dannce'], '1'),
    # pc.not_equal(table['sync'], '3'),
    pc.equal(table['test'], '0')
)
# Apply the filter and print the results
# for_dappy = table.filter(filter_mask) #filter_mask
socail = table.filter(pc.equal(table['social'], '1'))
# Print each row of the filtered table
print(socail.to_pandas())  # Display the filtered data in a familiar pandas-like format


   mir_generate_param sync dropf_handle com social miniscope test  \
0                   1    1            0   0      1         1    1   
1                   1    1            0   0      1         1    1   
2                   1    1            0   0      1         1    1   
3                   1    1            0   0      1         1    0   
4                   1    1            0   0      1         1    0   
5                   1    1            0   0      1         1    0   
6                   1    1            0   0      1         1    0   
7                   1    1            0   0      1         1    0   
8                   1    1            0   0      1         1    0   
9                   1    1            0   0      1         1    0   
10                  1    1            0   0      1         1    0   
11                  1    1            0   0      1         1    0   
12                  1    1            0   0      1         1    0   
13                  1    1        

In [6]:
import pyarrow.compute as pc
table = all_df  # combined_df
filter_mask = pc.and_(
    # pc.equal(table['mir_generate_param'], '1'),
    pc.equal(table['dannce'], '1'),
    # pc.not_equal(table['sync'], '3'),
    pc.equal(table['test'], '0')
)
# Apply the filter and print the results
# for_dappy = table.filter(filter_mask) #filter_mask
socail = table.filter(pc.equal(table['after_oxytocin'], '1'))
# Print each row of the filtered table
print(socail.to_pandas())  # Display the filtered data in a familiar pandas-like format


  mir_generate_param sync dropf_handle com social miniscope test  \
0                  1    1            0   1      0         1    0   
1                  1    1            0   1      0         1    0   
2                  1    1            0   0      1         1    0   
3                  1    1            0   1      1         1    0   
4                  1    3            3   0      0         1    0   
5                  1    1            0   1      0         1    0   
6                  1    1            0   0      1         1    0   

  after_oxytocin before_oxytocin dannce  \
0              1               0      1   
1              1               0      1   
2              1               0      0   
3              1               0      1   
4              1               0      0   
5              1               0      1   
6              1               0      0   

                                     rec_file                   scan_time  \
0                       20240819V

In [10]:
import os
import re
import csv
import pandas as pd  # Required for reading metadata.csv
from datetime import datetime

# Assuming for_dappy is your PyArrow Table and base_folder is defined

data_list = []

# Convert PyArrow Table to list of dictionaries
rows = for_dappy.to_pylist()

def clean_animal_id(rec_file):
    """Extract and clean the AnimalID from rec_file."""
    cleaned = re.sub(r'_(test|cricket|acrylic|hole|cyliner|left|right|AO|BO|single|dead|all|cylinder|righthole|lefthole).*', '', rec_file, flags=re.IGNORECASE)
    cleaned = re.sub(r'_\d{1,2}_\d{2}(_.*)?$', '', cleaned)
    cleaned = re.sub(r'_V(\d+)_r(\d+)', r'V\1r\2', cleaned)
    cleaned = re.sub(r'NMCFS', '_no_hole', cleaned)
    return cleaned

def extract_time(rec_file):
    """Extract time in HH:MM format from rec_file."""
    time_match = re.search(r'_(\d{2})_(\d{2})$', rec_file)
    if time_match:
        hour, minute = time_match.groups()
        return f"{hour}:{minute}"
    return ''

def extract_recording_time(videos_path):
    """Extract the recording time from metadata.csv."""
    metadata_path = os.path.join(videos_path, "Camera3", "metadata.csv")
    if not os.path.exists(metadata_path):
        return None
    try:
        # Read the metadata.csv file
        metadata = pd.read_csv(metadata_path, header=None, names=['Key', 'Value'])
        
        # Ensure the structure is consistent
        if 'totalTime' in metadata['Key'].values:
            # Extract the value associated with 'totalTime'
            total_time = metadata.loc[metadata['Key'] == 'totalTime', 'Value'].values[0]
            return float(total_time)
        else:
            print(f"'totalTime' column not found in {metadata_path}. Keys available: {list(metadata['Key'].values)}")
    except Exception as e:
        print(f"Error reading metadata.csv in {videos_path}: {e}")
    return "N/A"



def is_true(value):
    """Check if the value logically translates to True."""
    try:
        return float(value) == 1.0
    except (ValueError, TypeError):
        return False

for row in rows:
    date_folder = row['date_folder']
    rec_file = row['rec_file']
    miniscope = row.get('miniscope', '0')
    after_oxytocin = row.get('after_oxytocin', '0')
    before_oxytocin = row.get('before_oxytocin', '0')
    social = row.get('social', '0')
    habituation = row.get('habituation', '0')
    saline = row.get('saline', '0')
    test = row.get('test', '0')
    caffeine = row.get('caffeine', '0')

    # Check if cricket exists in the name
    cricket = '1' if 'cricket' in rec_file.lower() else '0'

    # Prediction path
    Prediction_path = f"{base_folder}/{date_folder}/{rec_file}/DANNCE/predict00/save_data_AVG.mat"

    # Extract AnimalID and Time
    AnimalID = clean_animal_id(rec_file)
    time_formatted = extract_time(rec_file)

    # If time_formatted is empty, get time from videos folder
    if not time_formatted:
        videos_path = f"{base_folder}/{date_folder}/{rec_file}/videos"
        if os.path.exists(videos_path):
            mod_time = os.path.getmtime(videos_path)
            dt = datetime.fromtimestamp(mod_time)
            time_formatted = dt.strftime('%H:%M')
        else:
            time_formatted = ''

    # Extract recording time from metadata.csv
    videos_path = f"{base_folder}/{date_folder}/{rec_file}/videos"
    recording_time = extract_recording_time(videos_path)

    # Date
    date = date_folder

    # Sex
    Sex = 'male'

    # Collect applicable conditions
    conditions = []
    if is_true(test):
        conditions.append('test')
    if is_true(miniscope):
        conditions.append('miniscope')
    if is_true(social):
        conditions.append('social')
    if is_true(after_oxytocin):
        conditions.append('after_oxytocin')
    if is_true(before_oxytocin):
        conditions.append('before_oxytocin')
    if is_true(habituation):
        conditions.append('habituation')
    if is_true(saline):
        conditions.append('saline')
    if is_true(caffeine):
        conditions.append('caffeine')
    if is_true(cricket):
        conditions.append('cricket')

    # Join conditions into a single string
    Condition = ','.join(conditions) if conditions else 'baseline'

    # Baseline logic
    baseline = '1' if 'baseline' in Condition else '0'

    # Append data row
    data_list.append({
        'AnimalID': AnimalID,
        'Sex': Sex,
        'Condition': Condition,
        'date': date,
        'time': time_formatted,
        'Prediction_path': Prediction_path,
        'miniscope': miniscope,
        'after_oxytocin': after_oxytocin,
        'before_oxytocin': before_oxytocin,
        'social': social,
        'habituation': habituation,
        'saline': saline,
        'test': test,
        'caffeine': caffeine,
        'cricket': cricket,
        'baseline': baseline,
        'recording_time': recording_time if recording_time else 'N/A'
    })

# Save to CSV
csv_columns = ['AnimalID', 'Sex', 'Condition', 'date', 'time', 'Prediction_path', 
               'miniscope', 'after_oxytocin', 'before_oxytocin', 'social', 
               'habituation', 'saline', 'test', 'caffeine', 'cricket', 
               'baseline', 'recording_time']

csv_file = os.path.join('/home/lq53/mir_repos/dappy_24_nov/mir_modif_dappy/meta_datas', '250429' + os.path.basename(base_folder) + '.csv')

try:
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        writer.writerows(data_list)
    print(f"CSV file saved to {csv_file}")
except IOError:
    print("I/O error occurred while writing the CSV file.")

CSV file saved to /home/lq53/mir_repos/dappy_24_nov/mir_modif_dappy/meta_datas/250429Oct3V1.csv


In [ ]:
# import pyarrow as pa
# import os
# from datetime import datetime
# import re
# import csv

# # Assuming for_dannce_vis is your PyArrow Table and base_folder is defined

# data_list = []

# # Convert the PyArrow Table to a list of dictionaries
# rows = for_dappy.to_pylist()

# def extract_animalid_and_time(rec_file):
#     parts = rec_file.split('_')
#     # Remove 'test' if it is the last part
#     if parts[-1].lower() == 'test':
#         parts.pop()
#     time_formatted = ''
#     # Check for time at the end
#     if len(parts) >= 1:
#         last_part = parts[-1]
#         # Match 'HHMM' format
#         if re.match(r'^\d{4}$', last_part):
#             time_str = parts.pop()
#             time_formatted = f"{time_str[:2]}:{time_str[2:]}"
#         # Match 'HH_MM' format
#         elif re.match(r'^\d{1,2}$', last_part) and len(parts) >= 2 and re.match(r'^\d{1,2}$', parts[-2]):
#             minute = parts.pop()
#             hour = parts.pop()
#             time_formatted = f"{hour.zfill(2)}:{minute.zfill(2)}"
#         else:
#             # Check if the last 4 characters are digits (time in 'HHMM' format without underscores)
#             if rec_file[-4:].isdigit():
#                 time_str = rec_file[-4:]
#                 time_formatted = f"{time_str[:2]}:{time_str[2:]}"
#                 animalid = rec_file[:-4]
#                 return animalid, time_formatted
#     animalid = '_'.join(parts)
#     return animalid, time_formatted

# for row in rows:
#     date_folder = row['date_folder']
#     rec_file = row['rec_file']
#     social = row.get('social', '0')
#     miniscope = row.get('miniscope', '0')
#     test = row.get('test', '0')
#     after_oxytocin = row.get('after_oxytocin', '0')
#     before_oxytocin = row.get('before_oxytocin', '0')

#     # Prediction_path
#     Prediction_path = f"{base_folder}/{date_folder}/{rec_file}/DANNCE/predict00/save_data_AVG.mat"

#     # Extract AnimalID and Time
#     AnimalID, time_formatted = extract_animalid_and_time(rec_file)

#     # If time_formatted is empty, attempt to get time from 'videos' folder
#     if not time_formatted:
#         videos_path = f"{base_folder}/{date_folder}/{rec_file}/videos"
#         if os.path.exists(videos_path):
#             mod_time = os.path.getmtime(videos_path)
#             dt = datetime.fromtimestamp(mod_time)
#             time_formatted = dt.strftime('%H:%M')
#         else:
#             time_formatted = ''

#     # Date
#     date = date_folder

#     # Sex
#     Sex = 'male'

#     # Collect applicable conditions
#     def is_true(value):
#         try:
#             return float(value) == 1.0
#         except (ValueError, TypeError):
#             return False

#     conditions = []
#     if is_true(test):
#         conditions.append('test')
#     if is_true(miniscope):
#         conditions.append('miniscope')
#     if is_true(social):
#         conditions.append('social')
#     if is_true(after_oxytocin):
#         conditions.append('after_oxytocin')
#     if is_true(before_oxytocin):
#         conditions.append('before_oxytocin')

#     # Join conditions into a single string
#     Condition = ','.join(conditions) if conditions else 'baseline'

#     # Collect data
#     data_list.append({
#         'AnimalID': AnimalID,
#         'Sex': Sex,
#         'Condition': Condition,
#         'date': date,
#         'time': time_formatted,
#         'Prediction_path': Prediction_path
#     })

# # Save to CSV
# csv_columns = ['AnimalID', 'Sex', 'Condition', 'date', 'time', 'Prediction_path']
# csv_file = os.path.join('/hpc/group/tdunn/lq53/dappy_24_nov/mir_modif_dappy/meta_datas', os.path.basename(base_folder) + '.csv')

# try:
#     with open(csv_file, 'w', newline='') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#         writer.writeheader()
#         writer.writerows(data_list)
#     print(f"CSV file saved to {csv_file}")
# except IOError:
#     print("I/O error occurred while writing the CSV file.")


CSV file saved to /hpc/group/tdunn/lq53/dappy_24_nov/mir_modif_dappy/meta_datas/Oct3V1.csv
